In [49]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.1.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:13 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:14 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:15 http://ppa.launchpad.net/graph

In [50]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2021-09-27 16:00:09--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.1’

postgresql-42.2.9.j 100%[===================>] 892.61K  --.-KB/s    in 0.08s   

2021-09-27 16:00:10 (10.4 MB/s) - ‘postgresql-42.2.9.jar.1’ saved [914037/914037]



In [51]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [52]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Watches_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
watches_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Watches_v1_00.tsv.gz"), sep="\t", header=True)
watches_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|    3653882|R3O9SGZBVQBV76|B00FALQ1ZC|     937001370|Invicta Women's 1...|         Watches|          5|            0|          0|   N|                Y|          Five Stars|Absolutely love t...| 2015-08-31|
|         US|   14661224| RKH8BNC3L5DLF|B00D3RGO20|     484010722|Kenneth Cole New ...|         Watches|          5|    

Drop na and duplicates

In [53]:
watches_df = watches_df.dropna()
watches_df = watches_df.dropDuplicates()

Schema - Watches_df

In [54]:
watches_df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: string (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: string (nullable = true)
 |-- helpful_votes: string (nullable = true)
 |-- total_votes: string (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)



Recast customer_id, product_parent, star_rating, helpful_votes, total_votes as IntegerType

In [55]:
from pyspark.sql.types import IntegerType
watches_df = watches_df.withColumn("customer_id",watches_df["customer_id"].cast(IntegerType()))
watches_df = watches_df.withColumn("product_parent",watches_df["product_parent"].cast(IntegerType()))
watches_df = watches_df.withColumn("star_rating",watches_df["star_rating"].cast(IntegerType()))
watches_df = watches_df.withColumn("helpful_votes",watches_df["helpful_votes"].cast(IntegerType()))
watches_df = watches_df.withColumn("total_votes",watches_df["total_votes"].cast(IntegerType()))

In [56]:
print(watches_df.select("customer_id").dtypes)
print(watches_df.select("product_parent").dtypes)
print(watches_df.select("star_rating").dtypes)
print(watches_df.select("helpful_votes").dtypes)
print(watches_df.select("total_votes").dtypes)

[('customer_id', 'int')]
[('product_parent', 'int')]
[('star_rating', 'int')]
[('helpful_votes', 'int')]
[('total_votes', 'int')]


In [57]:
watches_df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)



Count the number of records in the Database

In [58]:
# Row Count
watches_df.count()

960679

Transform the dataset to fit the tables in the schema file

In [59]:
from pyspark.sql.functions import to_date
# DataFrame
review_id = watches_df.select(["review_id", "customer_id", "product_id", "product_parent", to_date("review_date", 'yyyy-MM-dd').alias("review_date")])
review_id.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R101G33FIEDHQG|   22594238|B000PSN090|     425533441| 2015-02-08|
|R10339L4K4LX3C|   20301275|B00LTQTH58|     732935288| 2014-11-18|
|R1073CM97QGOPP|   32190651|B004V23YV0|     921921436| 2015-07-06|
|R107KYD2GYG4DK|    1424324|B004Y4CMCM|     452126791| 2014-07-05|
|R10AYHA3991CS9|   26642389|B00J0SSATQ|     682900500| 2014-10-28|
|R10C59FCUFOZPG|   15168495|B005FMYSWO|     824130779| 2012-03-07|
|R10DZLWV4QYCPI|    2239995|B000FVC5LG|     918169894| 2014-08-21|
|R10E5DEJ235Z19|    5378758|B003IHFGBK|     857934958| 2014-02-08|
|R10E6QP8RITUIA|    8495726|B00DI315GE|     734054846| 2015-03-23|
|R10F6W35V8C1ER|   25578677|B006076T5M|     832352606| 2013-04-10|
|R10G5F537HMGLR|   12806455|B00CJXMLNK|     908427132| 2014-01-06|
|R10GL8NCNQU8M7|   15231446|B00I9I70BM|     419690985| 2015-04

In [60]:
from pyspark import SparkFiles
# DataFrame
products = watches_df.select(["product_id", "product_title"])
products = products.drop_duplicates(["product_id", "product_title"])
products.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00NL1CZ1C|Nuband. Activ+ Ac...|
|B009BEO81I|        Fossil Riley|
|B000UMI0FK|Armitron Sport Wo...|
|B002NSKV6A|Kenneth Cole New ...|
|B0083XFT5M|Timex Unisex T2N8...|
|B00N1Y8TQ4|Tissot Men's T095...|
|B008B39MTI|XOXO Women's XO55...|
|B00CPJWMZK|Royal Crown Women...|
|B0013HSXXK|Timex Men's T5K08...|
|B004GHLE4Y|Timex Women's Ind...|
|B00GQFVBKQ|Double Watch Wind...|
|B00D3DUT48|Timex Women's T2P...|
|B00CBPLWMC|JOINNEW@ 2013 New...|
|B00J6AGSTM|Casio Women's BA-...|
|B003DQRWM2|Casio G-shock BG1...|
|B001O6K4DM| Single Watch Winder|
|B002YK5W5W|Disney Women's MK...|
|B004N49G5O|Nixon Rubber Re-R...|
|B00NJNAS4I|Digital Watches, ...|
|B00068TJJY|Seiko Men's SKA18...|
+----------+--------------------+
only showing top 20 rows



In [61]:
from pyspark import SparkFiles
# DataFrame
customers = watches_df.groupby(["customer_id"]).count()
customers = customers.withColumnRenamed("count", "customer_count")
customers.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|    2252188|             1|
|   52373157|             3|
|    2594156|             2|
|   13373248|             1|
|   20671851|            24|
|   45153737|             5|
|    2891243|             2|
|   45938967|             1|
|   39355523|             1|
|   29954480|             1|
|   45726037|             2|
|   12665682|             1|
|   12550672|             7|
|   22426070|             1|
|   13309307|             1|
|   11441161|             2|
|   25694626|             1|
|   12799553|             1|
|   24740812|             1|
|   12536485|             3|
+-----------+--------------+
only showing top 20 rows



In [62]:
from pyspark import SparkFiles
vine_table = watches_df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine"])
vine_table.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R101G33FIEDHQG|          3|            0|          0|   N|
|R10339L4K4LX3C|          4|            1|          2|   N|
|R1073CM97QGOPP|          5|            0|          0|   N|
|R107KYD2GYG4DK|          5|            0|          0|   N|
|R10AYHA3991CS9|          5|            2|          2|   N|
|R10C59FCUFOZPG|          5|            3|          3|   N|
|R10DZLWV4QYCPI|          5|            0|          0|   N|
|R10E5DEJ235Z19|          4|            1|          2|   N|
|R10E6QP8RITUIA|          5|            0|          0|   N|
|R10F6W35V8C1ER|          5|            0|          0|   N|
|R10G5F537HMGLR|          3|           26|         34|   N|
|R10GL8NCNQU8M7|          3|            0|          0|   N|
|R10LINWDMEJ36W|          5|            0|          0|   N|
|R10LY9CC9X0LFU|          5|            

Load the Dataframes that correspond to tables into an RDS instance

In [63]:
mode = 'append'
jdbc_url = "jdbc:postgresql://mypostgresdb.ceomfsf1omq9.us-east-2.rds.amazonaws.com:5432/my_data_class_db"
config = {
    'user':'postgres',
    'password':'postgres',
    'driver':"org.postgresql.Driver"
}

In [66]:
review_id.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [65]:
products.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [67]:
customers.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [68]:
vine_table.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)